In [1]:
Dataset_url = "https://www.kaggle.com/datasets/jehanbhathena/weather-dataset"

In [2]:
# Upload Kaggle json file
!mkdir ~/.kaggle/
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d jehanbhathena/weather-dataset

Dataset URL: https://www.kaggle.com/datasets/jehanbhathena/weather-dataset
License(s): CC0-1.0
100% 585M/587M [00:05<00:00, 129MB/s] 
100% 587M/587M [00:05<00:00, 117MB/s]


In [4]:
!unzip -q weather-dataset.zip

In [5]:
# Import
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import datasets, transforms, models
from PIL import Image
from tqdm.auto import tqdm
import time

In [6]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
device

'cuda'

In [7]:
# Define transformation for data augmentation or preprocessing
transform = transforms.Compose([
    transforms.Resize((255, 255)),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [8]:
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []

        # Gather file paths of all images in the folders and their corresponding labels
        for label, folder in enumerate(os.listdir(root_dir)):
            folder_path = os.path.join(root_dir, folder)
            if os.path.isdir(folder_path):
                for file in os.listdir(folder_path):
                  if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.webp')):
                    self.samples.append((os.path.join(folder_path, file), label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        try:
            image_path, label = self.samples[index]
            image = Image.open(image_path).convert('RGB')

            if self.transform:
                image = self.transform(image)

            return image, label

        except Exception as e:
            # If there's an error opening or processing the image, print the error and return None
            #print(f"Error loading image: {image_path} - {e}")
            return self.__getitem__(index + 1)

In [9]:
# Instantiate custom dataset
root_dir = '/content/dataset'
custom_dataset = CustomImageDataset(root_dir, transform=transform)

In [10]:
len(custom_dataset)

6862

In [11]:
# Define the size of the training set
train_size = int(0.8 * len(custom_dataset))
test_size = len(custom_dataset) - train_size

# Split the dataset into training and testing sets
train_dataset, test_dataset = torch.utils.data.random_split(custom_dataset, [train_size, test_size])

len(train_dataset), len(test_dataset)

(5489, 1373)

In [12]:
# Define batch size for DataLoader
batch_size = 16

# Create DataLoader for training set
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last = True)

# Create DataLoader for testing set
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [13]:
def train(dataloader, model, loss_fn, optimizer,device):
    size = len(dataloader.dataset)
    model.to(device)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [14]:
def test(dataloader, model, loss_fn, device):
    size = len(dataloader.dataset)
    model.to(device)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, input_shape, hidden_units, output_shape):
        super(ResNet, self).__init__()
        self.initial_conv = nn.Conv2d(in_channels=input_shape, out_channels=hidden_units, kernel_size=3, stride=1, padding=1)
        self.initial_bn = nn.BatchNorm2d(hidden_units)
        self.initial_relu = nn.ReLU()
        self.initial_pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.layer1 = self._make_layer(hidden_units, hidden_units, stride=1)
        self.layer2 = self._make_layer(hidden_units, hidden_units, stride=2)

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(156800, hidden_units * 10),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_units * 10),
            nn.Linear(hidden_units * 10, hidden_units * 5),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_units * 5),
            nn.Linear(hidden_units * 5, hidden_units),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_units),
            nn.Linear(hidden_units, output_shape),
            nn.Softmax(dim=1)
        )

    def _make_layer(self, in_channels, out_channels, stride):
        layer = ResidualBlock(in_channels, out_channels, stride)
        return layer

    def forward(self, x):
        x = self.initial_conv(x)
        x = self.initial_bn(x)
        x = self.initial_relu(x)
        x = self.initial_pool(x)

        x = self.layer1(x)
        x = self.layer2(x)

        x = self.classifier(x)
        return x

torch.manual_seed(23)
model = ResNet(input_shape=3, hidden_units=50, output_shape=11)


In [33]:
model

ResNet(
  (initial_conv): Conv2d(3, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (initial_bn): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (initial_relu): ReLU()
  (initial_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (layer1): ResidualBlock(
    (conv1): Conv2d(50, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(50, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (shortcut): Sequential()
  )
  (layer2): ResidualBlock(
    (conv1): Conv2d(50, 50, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (bn1): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(50, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNor

In [34]:
# Setup loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(),
                             lr=1e-4)

In [35]:
from tqdm.auto import tqdm
import time
start_time = time.time()


# Train and test model
epochs = 10
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n---------")
    train(train_loader, model, loss_fn, optimizer,device)
    test(test_loader, model, loss_fn,device)

current_time = time.time()
total = current_time - start_time
print("Done!")
print(f'Training Took: {total/60} minutes!')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0
---------
loss: 2.198267  [   16/ 5489]
loss: 2.157997  [ 1616/ 5489]
loss: 2.175851  [ 3216/ 5489]
loss: 2.164036  [ 4816/ 5489]
Test Error: 
 Accuracy: 48.2%, Avg loss: 2.153694 

Epoch: 1
---------
loss: 2.191469  [   16/ 5489]
loss: 2.217797  [ 1616/ 5489]
loss: 2.156900  [ 3216/ 5489]
loss: 2.141321  [ 4816/ 5489]
Test Error: 
 Accuracy: 50.5%, Avg loss: 2.142511 

Epoch: 2
---------
loss: 2.205796  [   16/ 5489]
loss: 2.179168  [ 1616/ 5489]
loss: 2.040957  [ 3216/ 5489]
loss: 2.105086  [ 4816/ 5489]
Test Error: 
 Accuracy: 49.8%, Avg loss: 2.137372 

Epoch: 3
---------
loss: 2.150480  [   16/ 5489]
loss: 2.230639  [ 1616/ 5489]
loss: 2.084796  [ 3216/ 5489]
loss: 2.238349  [ 4816/ 5489]
Test Error: 
 Accuracy: 50.8%, Avg loss: 2.132490 

Epoch: 4
---------
loss: 2.167538  [   16/ 5489]
loss: 2.197825  [ 1616/ 5489]
loss: 2.271698  [ 3216/ 5489]
loss: 2.167979  [ 4816/ 5489]
Test Error: 
 Accuracy: 51.0%, Avg loss: 2.129133 

Epoch: 5
---------
loss: 2.123002  [   16/ 54

In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResidualBlock_1(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock_1, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet_1(nn.Module):
    def __init__(self, input_channels, hidden_units, output_shape, num_blocks):
        super(ResNet_1, self).__init__()
        self.initial_conv = nn.Conv2d(in_channels=input_channels, out_channels=hidden_units, kernel_size=3, stride=1, padding=1)
        self.initial_bn = nn.BatchNorm2d(hidden_units)
        self.initial_relu = nn.ReLU()
        self.initial_pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.layer1 = self._make_layer(hidden_units, hidden_units, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(hidden_units, hidden_units*2, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(hidden_units*2, hidden_units*4, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(hidden_units*4, hidden_units*8, num_blocks[3], stride=2)

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(78400, hidden_units * 10),  # Adjusted flattened size
            nn.ReLU(),
            nn.BatchNorm1d(hidden_units * 10),
            nn.Linear(hidden_units * 10, hidden_units * 5),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_units * 5),
            nn.Linear(hidden_units * 5, hidden_units),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_units),
            nn.Linear(hidden_units, output_shape),
            nn.Softmax(dim=1)
        )

    def _make_layer(self, in_channels, out_channels, num_blocks, stride):
        layers = []
        layers.append(ResidualBlock_1(in_channels, out_channels, stride))
        for _ in range(1, num_blocks):
            layers.append(ResidualBlock_1(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.initial_conv(x)
        x = self.initial_bn(x)
        x = self.initial_relu(x)
        x = self.initial_pool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.classifier(x)
        return x

torch.manual_seed(23)
num_blocks = [3, 4, 23, 3]  # Example for a deeper network (similar to ResNet-101)
model_1 = ResNet_1(input_channels=3, hidden_units=50, output_shape=11, num_blocks=num_blocks)

# Print model summary
print(model_1)


ResNet_1(
  (initial_conv): Conv2d(3, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (initial_bn): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (initial_relu): ReLU()
  (initial_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResidualBlock_1(
      (conv1): Conv2d(50, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(50, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): ResidualBlock_1(
      (conv1): Conv2d(50, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(50, 50, kernel_size=(3, 3), stride=(

In [37]:
# Setup loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_1.parameters(),
                             lr=1e-3)

In [38]:
from tqdm.auto import tqdm
import time
start_time = time.time()


# Train and test model
epochs = 10
for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n---------")
    train(train_loader, model_1, loss_fn, optimizer,device)
    test(test_loader, model_1, loss_fn,device)

current_time = time.time()
total = current_time - start_time
print("Done!")
print(f'Training Took: {total/60} minutes!')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0
---------
loss: 2.419941  [   16/ 5489]
loss: 2.369678  [ 1616/ 5489]
loss: 2.322367  [ 3216/ 5489]
loss: 2.239841  [ 4816/ 5489]
Test Error: 
 Accuracy: 32.2%, Avg loss: 2.277165 

Epoch: 1
---------
loss: 2.233067  [   16/ 5489]
loss: 2.261565  [ 1616/ 5489]
loss: 2.125225  [ 3216/ 5489]
loss: 2.291066  [ 4816/ 5489]
Test Error: 
 Accuracy: 41.6%, Avg loss: 2.192803 

Epoch: 2
---------
loss: 2.207926  [   16/ 5489]
loss: 2.196465  [ 1616/ 5489]
loss: 2.268047  [ 3216/ 5489]
loss: 2.237510  [ 4816/ 5489]
Test Error: 
 Accuracy: 45.8%, Avg loss: 2.147407 

Epoch: 3
---------
loss: 2.047021  [   16/ 5489]
loss: 2.150836  [ 1616/ 5489]
loss: 2.296468  [ 3216/ 5489]
loss: 2.116715  [ 4816/ 5489]
Test Error: 
 Accuracy: 50.8%, Avg loss: 2.112111 

Epoch: 4
---------
loss: 2.201841  [   16/ 5489]
loss: 2.201419  [ 1616/ 5489]
loss: 2.154342  [ 3216/ 5489]
loss: 2.118582  [ 4816/ 5489]
Test Error: 
 Accuracy: 50.1%, Avg loss: 2.112994 

Epoch: 5
---------
loss: 2.179900  [   16/ 54